In [1]:
# !uv pip install langchain langchain-openai langchain-community faiss-cpu openai tiktoken trafilatura playwright nest_asyncio langchain_experimental
# !uv add langchain langchain-openai langchain-community faiss-cpu openai tiktoken trafilatura playwright nest_asyncio langchain_experimental
# !playwright install

In [5]:
from src.utils import get_config

config = get_config() 

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os

 
# กำหนด LLM เป็น Typhoon
typhoon_llm = ChatOpenAI(
    api_key=config['typhoon']['token'],
    base_url="https://api.opentyphoon.ai/v1",
    model="typhoon-v2.1-12b-instruct",
    temperature=0
)

embeddings = OpenAIEmbeddings(
    api_key=config['openai']['token'],
    model="text-embedding-3-small"
    ) 


import os
import time 
# --- ส่วนของการตั้งค่า Path ---
DATA_FOLDER = "data/raw"  # แก้ไขจาก "product_data" เป็น "data/raw"
FILE_NAMES = ["workshop.txt", "rerun.txt", "overall.txt"]
FAISS_INDEX_PATH = "artifacts/faiss_product_index"  # กำหนดชื่อโฟลเดอร์ Index

# สร้าง DATA_PATHS จาก DATA_FOLDER และ FILE_NAMES
DATA_PATHS = [os.path.join(DATA_FOLDER, fname) for fname in FILE_NAMES]

# --- ส่วนของการโหลด/สร้าง Index ---
index_file = os.path.join(FAISS_INDEX_PATH, "index.faiss")

if os.path.exists(index_file):
    print(f"\n✅ พบ Index ที่บันทึกไว้, กำลังโหลดจาก '{FAISS_INDEX_PATH}'...")
    start_time = time.time()
    vectorstore = FAISS.load_local(
        FAISS_INDEX_PATH,
        embeddings,
        allow_dangerous_deserialization=True
    )
    end_time = time.time()
    print(f"✅ โหลด Index สำเร็จใน {end_time - start_time:.2f} วินาที")
else:
    print(f"\n🟡 ไม่พบ Index, กำลังสร้าง Index ใหม่จากไฟล์: {', '.join(FILE_NAMES)}")
    start_time = time.time()
    all_docs = []
    # Loop นี้จะใช้ DATA_PATHS ที่เราเพิ่งสร้างด้านบน
    for path in DATA_PATHS:
        if not os.path.exists(path):
            print(f"‼️ ข้อผิดพลาด: ไม่พบไฟล์ที่ '{path}' กรุณาตรวจสอบตำแหน่งไฟล์")
            continue
        print(f"  - กำลังโหลดไฟล์: {path}")
        loader = TextLoader(path, encoding="utf-8")
        docs_from_file = loader.load()
        all_docs.extend(docs_from_file)

    # ตรวจสอบว่าโหลดไฟล์สำเร็จหรือไม่
    if not all_docs:
        print("\n‼️ ข้อผิดพลาดร้ายแรง: ไม่สามารถโหลดเอกสารใดๆ ได้เลย")
        print("กรุณาตรวจสอบ Path และโครงสร้างโฟลเดอร์ให้ถูกต้อง")
    else:
        print(f"\n✅ โหลดเอกสารทั้งหมด {len(all_docs)} document(s) เรียบร้อย")

        text_splitter = SemanticChunker(
            embeddings=embeddings,
            breakpoint_threshold_type="percentile"  # สามารถเลือกได้หลายแบบ เช่น "standard_deviation", "interquartile", "gradient"
        )
        splits = text_splitter.split_documents(all_docs)
        print(f"✅ แบ่งเอกสารออกเป็น {len(splits)} chunks")

        print("⏳ กำลังสร้าง Vector Store ด้วย FAISS (อาจใช้เวลาสักครู่)...")
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(FAISS_INDEX_PATH)
        end_time = time.time()
        print(f"💾 Index ถูกสร้างและบันทึกไว้ที่ '{FAISS_INDEX_PATH}' ใน {end_time - start_time:.2f} วินาที")

# สร้าง retriever จาก vectorstore (เพิ่มส่วนนี้เพื่อแก้ไข NameError)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # จำนวน chunks ที่จะดึงมา
)

NameError: name 'config' is not defined

In [13]:
template = """
### ROLE ###
คุณคือ AI Sales & Support Specialist

### OBJECTIVE ###
เป้าหมายของคุณคือการแปลงคำถามของลูกค้าให้กลายเป็นการขาย (Conversion) โดยการให้ข้อมูลที่ถูกต้อง, เน้นคุณประโยชน์, และสร้างประสบการณ์ที่น่าประทับใจ โดยต้องอ้างอิงจากข้อมูลในส่วน KNOWLEDGE BASE เท่านั้น

### TONE OF VOICE ###
- Professional (เป็นมืออาชีพ)
- Helpful (ช่วยเหลือ)
- Persuasive (โน้มน้าว)
- Confident (มั่นใจ)
- Thai language, formal "ครับ/ค่ะ" tone.

### WORKFLOW ###
1.  **Acknowledge and Understand:** ขึ้นต้นด้วยการตอบรับคำถามของลูกค้า และแสดงความเข้าใจในสิ่งที่เขาต้องการ
2.  **Extract & Reframe:** ดึงข้อมูลที่เกี่ยวข้องจาก KNOWLEDGE BASE จากนั้นนำเสนอในรูปแบบของ "ประโยชน์" ที่ลูกค้าจะได้รับ
3.  **Address Concerns & Compare:** หากลูกค้าแสดงความกังวลหรือต้องการเปรียบเทียบ ให้ใช้ข้อมูลเพื่อคลายความกังวลและชี้ให้เห็นจุดเด่นของสินค้าเรา
4.  **Suggest & Guide (Call-to-Action):** เสนอขั้นตอนต่อไปอย่างชัดเจน เช่น แนะนำให้ซื้อ, สอบถามข้อมูลการชำระเงิน, หรือให้ข้อมูลโปรโมชั่น

### RULES ###
- DO NOT invent information not present in the KNOWLEDGE BASE.
- If information is missing, politely state that you need to check with a specialist and offer to assist with other questions.
- Always end the conversation with a call to action or an offer for further help.

---
### KNOWLEDGE BASE (Context) ###
{context}

### CUSTOMER QUESTION (Question) ###
{question}

---
### RESPONSE ###
"""
prompt = ChatPromptTemplate.from_template(template)
 

# 4. สร้าง Chain (RAG Chain)
# คือการต่อท่อการทำงานทั้งหมดเข้าด้วยกัน
rag_chain = (
    RunnableParallel(
        context=retriever,
        question=RunnablePassthrough()
    )
    | prompt
    | typhoon_llm
    | StrOutputParser()
)

# 5. ลองถามคำถามผ่าน Chain
print("\n--- 🤖 เริ่มระบบถาม-ตอบ (RAG) ---")
question_to_ask = "มีเวิคช้อบอะไรบ้างครับ เเล้วขายยังไง"
answer = rag_chain.invoke(question_to_ask)

print(f"คำถาม: {question_to_ask}")
print(f"คำตอบจาก AI: {answer}")


--- 🤖 เริ่มระบบถาม-ตอบ (RAG) ---
คำถาม: มีเวิคช้อบอะไรบ้างครับ เเล้วขายยังไง
คำตอบจาก AI: สวัสดีครับ ยินดีต้อนรับสู่ SCB 10X ครับ

ผมเข้าใจว่าคุณกำลังสนใจเกี่ยวกับเวิร์คช็อปของเรานะครับ ตอนนี้เรามีเวิร์คช็อปที่น่าสนใจดังนี้ครับ:

*   **Online Workshops "จับมือเทรดด้วย Quant Data":** เมื่อคุณซื้อเวิร์คช็อปใด ๆ ราคา 3,599 บาท คุณจะได้รับสิทธิพิเศษมากมายครับ
    *   RERUN ทุก Session ทั้ง Stage 1 & 2 (รวม 11 ชั่วโมง)
    *   RERUN Crypto Quant Trading (รวม 7 ชั่วโมง)
    *   สิทธิ์ใช้เครื่องมือดู Quant Tool ของ Investic Analytics Studio เป็นระยะเวลา 1 เดือน
*   **Rerun Quant Offside 3x:** เนื้อหาครอบคลุมตั้งแต่การใช้ Market Breadth ในการ Filter ตลาด, การวิเคราะห์พฤติกรรมหุ้นที่แข็งกว่าตลาด, การประเมินมูลค่าหุ้นด้วย Fundamental Analysis ไปจนถึงการใช้ AI ช่วยในการวิเคราะห์ และอื่นๆ อีกมากมายครับ
*   **Mastering Crypto Trading: Essential Indicators, Tools, and Yield Strategies:** สอนการวิเคราะห์ตลาดคริปโตอย่างเป็นระบบ, การใช้เครื่องมือและข้อมูลเชิงลึก, และกลยุทธ์การสร้างผลตอบแทน (Yield) ที่

In [ ]:
from src.components.ingestion import DataIngestionPipeline 

# # 1. รัน Data Ingestion Pipeline
print("Starting data ingestion...")
ingestion = DataIngestionPipeline(model_config_path="configs/model_config.yaml", environment_config_path="config.yaml")
# vectorstore = ingestion.get_or_create_vectorstore( 
#     use_semantic_chunking=True,   
# ) 
# # 2. ตั้งค่า RAG Chain
# print("Setting up RAG chain...")
# rag_chain = RAGChain()

# # โหลด index และ texts จาก ingestion
# rag_chain.load_index_and_texts(texts=result["texts"])

# # สร้าง chain
# rag_chain.initialize_chain()

# # 3. ทดสอบ query
# while True:
#     query = input("\nEnter your query (or 'exit' to quit): ")
#     if query.lower() == "exit":
#         break
        
#     # ส่ง query ไปยัง RAG chain
#     response = rag_chain.query(query)
    
#     # แสดงผลลัพธ์
#     print("\nResponse:")
#     print(response["answer"])
    
#     # แสดงเอกสารที่ใช้ (optional)
#     print("\nRetrieved documents:")
#     for i, doc in enumerate(response["context"]):
#         print(f"\nDocument {i+1}:")
#         print(f"Content: {doc.page_content[:100]}...")
#         print(f"Score: {doc.metadata.get('score', 'N/A')}")


Starting data ingestion...
